In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
p279WordSimSeededDF_wabs_text = pd.read_csv("../data/P279_dataset/P279_ChildPar_19k_WEmbAndCosSim.csv")
p279Seeded_SiblingsDF3_wabs_text = pd.read_csv("../data/P279_dataset/P279_Siblings_19k_WEmbAndCosSim.csv")

In [3]:
p279WordSimSeededDF_wabs_text.head()

,node2,node1,id,node1_label,label,node2_label,sent,node1_emb,node2_emb,bert2SentSim,...,BERT_abstract_emb_right,BERT_abstract_firstSent_emb_right,textEmb_7props_left,textEmb_7props_right,textEmb_2props_left,textEmb_2props_right,textEmb_7props_cosSim,textEmb_2props_cosSim,BERT_abstract_cosSim,BERT_abstract_firstSent_cosSim
0,Q100171002,Q1503443,Q1503443-P279-Q100171002-8282ae74-0,secrecy,P279,concealment,secrecy is concealment,[ 4.83600467e-01 1.25958338e-01 1.84784138e+...,[ 3.61319035e-01 1.03393383e-01 1.50166345e+...,0.901134,...,NaN,NaN,"[0.87660795, 0.4991357, -0.39552155, 0.7046891...","[0.36156428, -0.015657336, -0.14483449, -0.253...","[0.79082793, 0.53679967, -0.3825465, 0.6778466...","[0.36156428, -0.015657336, -0.14483449, -0.253...",0.832602,0.823419,-1.000000,-1.000000
1,Q1002697,Q49850,Q49850-P279-Q1002697-0c52cf68-0,journal,P279,periodical,journal is periodical,[-5.87019742e-01 -3.76376987e-01 2.21003819e+...,[-2.23196611e-01 -4.97725368e-01 2.20271492e+...,0.863449,...,"[-0.28667202591896057, -0.4099476933479309, 0....","[-0.25435054302215576, -0.92085862159729, 1.68...","[1.256428, 0.3254261, -0.26960722, 0.101883665...","[0.92698413, 0.32396156, -0.7647308, 0.6973268...","[1.256428, 0.3254261, -0.26960722, 0.101883665...","[1.0344226, 0.39984593, -0.76180136, 0.8256311...",0.742638,0.734428,-1.000000,-1.000000
2,Q101998,Q4421,Q4421-P279-Q101998-d0983886-0,forest,P279,biome,forest is biome,[-4.80742343e-02 -4.51617390e-01 1.48896754e+...,[ 5.09670794e-01 -4.42380428e-01 1.53245163e+...,0.507148,...,"[0.12320789694786072, 0.2723684310913086, 0.05...","[0.5991100072860718, 0.15195603668689728, 0.59...","[0.8141036, 0.14491142, -0.59703183, 0.335001,...","[-0.16750671, -0.17960861, -0.58211946, -0.687...","[0.91628444, 0.104261845, -0.49683735, 0.21296...","[-0.13534825, -0.23398273, -0.5165668, -0.7297...",0.647031,0.707611,0.508060,0.455747
3,Q102165,Q2294,Q2294-P279-Q102165-0893a180-0,proton,P279,nucleon,proton is nucleon,[-3.39149237e-01 2.01575115e-01 1.08608294e+...,[-6.87960759e-02 -1.65659860e-01 9.53789592e-...,0.798707,...,"[0.17410606145858765, 0.6839145421981812, -0.8...","[-0.04499293863773346, 0.6921464204788208, -0....","[0.469377, 0.7162001, -0.36740896, 0.76742333,...","[-0.11307852, 0.65730673, -0.13959256, 1.46633...","[0.4080446, 0.7498908, -0.2665757, 0.74437106,...","[-0.11307852, 0.65730673, -0.13959256, 1.46633...",0.836531,0.847591,0.855959,0.770030
4,Q102205,Q11435,Q11435-P279-Q102205-7b448d05-0,liquid,P279,fluid,liquid is fluid,[-5.40423356e-02 -1.01775277e+00 2.21057415e+...,[ 1.30038410e-01 -8.53226840e-01 2.37821031e+...,0.934246,...,"[0.4393930733203888, 0.7159943580627441, 0.647...","[0.5091343522071838, -0.05845202878117561, 1.0...","[-0.19995248, 0.65605164, -0.14234662, 0.43931...","[0.4293771, 1.1976917, 0.04422909, 0.5975481, ...","[-0.45549682, 0.73478854, -0.1980287, 0.436793...","[0.33631665, 1.1661042, 0.13744242, 0.6776439,...",0.765589,0.780342,0.752693,0.515319


In [5]:
p279WordSimSeededDF_wabs_text.bert2SentSim.describe()

count    19454.000000
mean         0.602876
std          0.146181
min          0.064503
25%          0.493110
50%          0.597949
75%          0.711631
max          1.000000
Name: bert2SentSim, dtype: float64

# Retrofitting Setup

In [6]:
def normalize(embedDict):
    for key, val in embedDict.items():
        temp = np.array([float(val1) for val1 in val])
        temp2 = temp**2
        embedDict[key] = temp / np.sqrt((temp2.sum() + 1e-6))
    return embedDict

In [7]:
def fetchEmbeddings(df):
    embedDict = {}
    for _, row in df.iterrows():
        embedDict[row.node] = row.value
    return normalize(embedDict)

In [8]:
def fetchNeighbours(df):
    neighboursDict = {}
    for _, row in df.iterrows():
        if row.node1_label not in neighboursDict:
            neighboursDict[row.node1] = []
        neighboursDict[row.node1].append((row.node2, row.bert2SentSim))
        
        if row.node2_label not in neighboursDict:
            neighboursDict[row.node2] = []
        neighboursDict[row.node2].append((row.node1, row.bert2SentSim))
    return neighboursDict

In [9]:
def retrofit(embedDict, neighDict):
    newEmbedDict = {}
    for word in embedDict.keys():
        if word in neighDict:
            neighbs = neighDict[word]
            neighbs = list(filter(lambda p: p[0] in embedDict, neighbs))
            if len(neighbs) == 0:
                newEmbedDict[word] = embedDict[word]
                continue
            sumOfSims = sum([neighb[1] for neighb in neighbs])
            sumOfEmbs = sum([embedDict[neighb[0]] * float(neighb[1]) for neighb in neighbs])
    #         newEmbedDict[word] = (embedDict[word] * (len(neighbs)) + sum([embedDict[neighb[0]] * float(neighb[1]) for neighb in neighbs])) / ((len(neighbs)) + sum([neighb[1] for neighb in neighbs]))
            newEmbedDict[word] = (embedDict[word] * (len(neighbs))**2 + sumOfEmbs) / ((len(neighbs))**2 + sumOfSims)
    #         newEmbedDict[word] = (embedDict[word] * (len(neighbs))**0.5 + sum([embedDict[neighb[0]] * float(neighb[1]) for neighb in neighbs])) / ((len(neighbs))**0.5 + sum([neighb[1] for neighb in neighbs]))
        else:
            newEmbedDict[word] = embedDict[word]
    return newEmbedDict

In [10]:
def determineDistances(embedDict, newEmbedDict):
    dist = []
    for word in embedDict.keys():
        dist.append(euclidean_distances([embedDict[word]], [newEmbedDict[word]])[0][0])
    return dist

# Attempts

## Concatenated ChildPar + Siblings Dataset

### Text Embeddings | 19k + 19k dataset

In [171]:
textEmb_7props_DF = pd.read_csv('../data/P279_dataset/output/P279-text-embedding-7-props-all.tsv', sep='\t')

In [173]:
textEmb_7props_DF['value'] = textEmb_7props_DF['value'].apply(lambda p: [float(p1) for p1 in p.split(',')])

In [174]:
textEmb_7props_DF.head()

,node,property,value
0,Q99907279,text_embedding,"[-0.13148594, 0.20147827, 0.015393771, 1.12779..."
1,Q99898510,text_embedding,"[0.19386396, 0.41306746, -0.32783666, 0.450188..."
2,Q99526025,text_embedding,"[0.89479333, 0.27038768, 0.15989815, -0.092144..."
3,Q99365546,text_embedding,"[0.10686234, 0.95048314, 0.32389534, 0.1960084..."
4,Q993308,text_embedding,"[0.23211154, 0.060303785, -1.0003253, 0.637829..."


In [176]:
embedDict = fetchEmbeddings(textEmb_7props_DF)

In [177]:
neighDict = fetchNeighbours(pd.concat([p279WordSimSeededDF_wabs_text, p279Seeded_SiblingsDF3_wabs_text]))

In [178]:
newEmbedDict = retrofit(embedDict, neighDict)

In [179]:
len(embedDict.keys())

19155

In [180]:
dists = determineDistances(embedDict, newEmbedDict)

In [181]:
pd.Series(dists).describe()

count    19155.000000
mean         0.310037
std          0.062412
min          0.000000
25%          0.269865
50%          0.310933
75%          0.351558
max          0.564922
dtype: float64